In [28]:
# Import all the libraries
import pandas as pd
import glob
import os
import re
from pprint import pprint

In [29]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\AZ\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'general'])
]


# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'primary','president'])
]


In [30]:
print("General files:")
for f in general_files:
    print(f)


General files:
C:\Huy Phan\College\VoterTurnout\data\AZ\20161108__az__general__apache__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20161108__az__general__cochise__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20161108__az__general__coconino__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20161108__az__general__graham__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20161108__az__general__greenlee__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20161108__az__general__la_paz__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20161108__az__general__maricopa__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20161108__az__general__mohave__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20161108__az__general__navajo__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20161108__az__general__pima__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20161108__az__general__pinal__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20161

In [31]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:
C:\Huy Phan\College\VoterTurnout\data\AZ\20160322__az__primary__president__apache__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20160322__az__primary__president__cochise__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20160322__az__primary__president__coconino__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20160322__az__primary__president__gila__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20160322__az__primary__president__greenlee__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20160322__az__primary__president__la_paz__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20160322__az__primary__president__maricopa__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20160322__az__primary__president__mohave__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20160322__az__primary__president__pima__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\AZ\20160322__az__primary__president__pinal__precinct.csv
C:\Huy Phan\College\Vo

In [32]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)
        # Create county name column
        parts = re.split(r'(?:__|[\\/\.])', file)


        county_name = parts[-3].upper()  
        # Add county column
        df["county"] = county_name
        df["county"] = df["county"].str.upper()
        # Exclude counties that have not county + precinct
        if 'precinct_name' not in df.columns:
            df = pd.DataFrame()  # remove the county

        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct_name"].astype(str)
        df["precinct"] = df["precinct"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
        
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"] == "President"]
        
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)

# Checking any suspicious precinct
suspicious_precincts = pri_combined_df[
    pri_combined_df["precinct"].str.strip().str.upper().isin(["TOTAL", "NAN"])
]
pri_combined_df


Error in C:\Huy Phan\College\VoterTurnout\data\AZ\20160322__az__primary__president__coconino__precinct.csv: 'county'
Error in C:\Huy Phan\College\VoterTurnout\data\AZ\20160322__az__primary__president__gila__precinct.csv: 'county'


,Unnamed: 0,contest,precinct_name,contest_id,contest_name,choice,choice_name,number,county,precinct,...,processed_done,processed_started,contest_total,write_in,undervote,overvote,Unnamed: 0.1,precinct_name.1,precinct_designation,late_early_votes
0,0.0,20,ALPINE,999999.0,Race Statistics,999999.0,Number of Precincts,1.0,APACHE,APACHEALPINE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,20,ALPINE,999999.0,Race Statistics,999998.0,Number of Precincts Reporting,1.0,APACHE,APACHEALPINE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,20,ALPINE,10.0,President - Dem,999999.0,Number of Precincts for Race,1.0,APACHE,APACHEALPINE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,20,ALPINE,10.0,President - Dem,999998.0,Number of Precincts Reporting,1.0,APACHE,APACHEALPINE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,20,ALPINE,10.0,President - Dem,999997.0,Registered Voters,80.0,APACHE,APACHEALPINE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15881,137.0,999999,TOTAL,60.0,President - Grn CD4,999998.0,Number of Precincts Reporting,1.0,YUMA,YUMATOTAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15882,138.0,999999,TOTAL,60.0,President - Grn CD4,999997.0,Registered Voters,7.0,YUMA,YUMATOTAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15883,139.0,999999,TOTAL,60.0,President - Grn CD4,999996.0,Times Counted,0.0,YUMA,YUMATOTAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15884,140.0,999999,TOTAL,60.0,President - Grn CD4,10.0,"Mesplay, Kent",0.0,YUMA,YUMATOTAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# cleaning 

In [33]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "votes"]]

primary_data

KeyError: "['party', 'candidate', 'votes'] not in index"

In [487]:
# Viewing candidate data
primary_data["candidate"].value_counts(dropna=False)

candidate
Hillary Clinton       5894
Bernie Sanders        5894
Keith Judd            5890
Star Locke            5889
Martin J. O'Malley    5860
                      ... 
Fiorina, Carly           4
Cruz, Ted                4
Christie, Chris          4
Bush, Jeb                4
Carson, Ben              4
Name: count, Length: 83, dtype: int64

In [488]:
# Cleaning Candidates

# Turning all primary data to uppercase
primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()

unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*"
]

pattern = "|".join(unwanted_keywords)

# Assuming candidate column is already string and uppercase
primary_data = primary_data[~primary_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

# Fixing the , candidate
primary_data["candidate"] = (
    primary_data["candidate"].str.split(",")
    .str[0]
    .str.strip()
    )

# Fixing De la Fuente
primary_data.loc[
    primary_data["candidate"].str.contains("\w*Fuen\w*", case=False, na=False),
    "candidate"
] = "LA FUENTE"

#Selecting only last name
primary_data["candidate"] = primary_data["candidate"].str.split().str[-1]

# Fixing Huckabee
primary_data.loc[
    primary_data["candidate"].str.contains("\w*ABEE\w*", case=False, na=False),
    "candidate"
] = "HUCKABEE"

# Fixing Fiorina
primary_data.loc[
    primary_data["candidate"].str.contains("\w*ORINA\W*", case=False, na=False),
    "candidate"
] = "FIORINA"

# Fixing Gray
primary_data.loc[
    primary_data["candidate"].str.contains("GREY", case=False, na=False),
    "candidate"
] = "GRAY"

# Fixing Wilson
primary_data.loc[
    primary_data["candidate"].str.contains("WISON", case=False, na=False),
    "candidate"
] = "WILSON"


# Fixing separator 
# primary_data["candidate"] = (
#     primary_data["candidate"]
#     .str.split(r"\s*(?:and|/|&|–|-|\+)\s*", n=1, expand=True)[0]
#     .str.strip()
#     .str.upper()
# )

# # Fixing McMullin

# primary_data.loc[
#     (
#         primary_data["candidate"].str.contains("MCMULLIN", case=False, na=False) |
#         primary_data["candidate"].str.contains("EVAN MCMULLEN", case=False, na=False)
#     ),
#     "candidate"
# ] = "MCMULLIN"

primary_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\2713288584.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\2713288584.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = (
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\2713288584.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

candidate
CLINTON     5937
SANDERS     5937
WILSON      5933
JUDD        5933
HAWES       5933
O'MALLEY    5933
FUENTE      5933
LOCKE       5931
TRUMP       5917
CRUZ        5916
RUBIO       5916
CARSON      5915
KASICH      5912
BUSH        5912
GRAHAM      5911
CHRISTIE    5910
FIORINA     5910
GRAY        5910
PAUL        5910
HUCKABEE    5910
SANTORUM    5910
Name: count, dtype: int64

In [489]:
# Viewing Party
primary_data["party"].value_counts(dropna=False)

party
REP                 73510
DEM                 45216
REPUBLICAN PARTY     1750
DEMOCRATIC PARTY     1142
R                     819
Republican            624
D                     504
Democratic            384
NaN                   380
Name: count, dtype: int64

In [490]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown
    

#=====================================
# Function to fill remaining missing party from master lookup
#=====================================
def fill_party_from_master(df, master_df):
    party_map = master_df.set_index("candidate")["party"].to_dict()
    df["party"] = df.apply(
        lambda row: party_map.get(row["candidate"], row["party"])
        if pd.isna(row["party"]) else row["party"],
        axis=1
    )
    return df


#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)
    
    # Combine unique pairs
    new_data = df[["candidate", "party"]].dropna().drop_duplicates()
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
primary_data["party"] = primary_data.apply(
    lambda row: fill_party_from_data(row, primary_data),
    axis=1
)

# Fill remaining party using general master CSV
# master_party_df = pd.read_csv("data/master_primary_candidate_party.csv")
# general_data = fill_party_from_master(general_data, master_party_df)

# STEP 3: Update master file with new (candidate, party) pairs
# update_master_candidate_party(general_data, "data/master_candidate_party.csv")
primary_data["party"].value_counts(dropna=False)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\509361827.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data.apply(


party
REP                 73666
DEM                 45440
REPUBLICAN PARTY     1750
DEMOCRATIC PARTY     1142
R                     819
Republican            624
D                     504
Democratic            384
Name: count, dtype: int64

In [491]:
# Cleaning Party
primary_data["party"] = primary_data.apply(
    lambda row: fill_party_from_data(row, primary_data),
    axis=1
)
primary_data.loc[:,"party"] = (
    primary_data["party"]
    .replace({
        "REPUBLICAN PARTY": "REP",
        "R": "REP",
        "Republican":"REP",
        "DEMOCRATIC PARTY": "DEM",
        "D": "DEM",
        "Democratic": "DEM"
    })
    .fillna("IND")
)
primary_data["party"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\2967241155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data.apply(


party
REP    76859
DEM    47470
Name: count, dtype: int64

In [492]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"]
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\2521670330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data.loc[:,"candidate_column"] = (


candidate_column,precinct,pri_dem_CLINTON,pri_dem_FUENTE,pri_dem_HAWES,pri_dem_JUDD,pri_dem_LOCKE,pri_dem_O'MALLEY,pri_dem_SANDERS,pri_dem_WILSON,pri_rep_BUSH,...,pri_rep_CRUZ,pri_rep_FIORINA,pri_rep_GRAHAM,pri_rep_GRAY,pri_rep_HUCKABEE,pri_rep_KASICH,pri_rep_PAUL,pri_rep_RUBIO,pri_rep_SANTORUM,pri_rep_TRUMP
0,ANDERSON1,54,0,0,0,0,0,20,0,10,...,258,0,0,0,4,16,3,72,1,143
1,ANDERSON10,13,0,0,0,0,0,3,0,1,...,103,0,0,0,0,1,0,18,0,56
2,ANDERSON11,51,0,0,0,0,0,12,0,2,...,193,0,0,0,0,7,1,31,0,98
3,ANDERSON13,65,0,0,0,0,1,18,0,8,...,250,1,1,1,1,15,0,57,0,158
4,ANDERSON15,20,0,0,0,0,0,13,0,0,...,190,0,0,0,0,10,1,41,0,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,YOUNG7,6,0,0,0,0,1,4,0,1,...,28,0,0,0,1,0,0,7,0,21
5998,ZAPATA1,597,34,2,6,5,7,223,10,0,...,11,0,0,0,0,0,0,2,0,7
5999,ZAPATA2,518,29,3,6,1,3,149,9,0,...,8,0,0,0,0,0,0,4,0,4
6000,ZAPATA3,440,40,5,5,1,7,195,7,1,...,12,0,0,0,1,0,0,5,0,22


In [493]:
# Process general files
general_df_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)
        
        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
        
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"] == "President"]
        
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        general_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(general_df_list, ignore_index=True)

# Checking any suspicious precinct
suspicious_precincts = gen_combined_df[
    gen_combined_df["precinct"].str.strip().str.upper().isin(["TOTAL", "NAN"])
]


In [494]:
# Select only the relevant columns
general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]

general_data

,precinct,party,candidate,votes
0,ANDERSON1,Rep,Donald J. Trump / Mike Pence,742
1,ANDERSON1,Dem,Hillary Clinton / Tim Kaine,262
2,ANDERSON1,Lib,Gary Johnson / William Weld,18
3,ANDERSON1,Grn,Jill Stein / Ajamu Baraka,5
4,ANDERSON1,NaN,Darrel L Castle,1
...,...,...,...,...
77388,ZAVALA2,Grn,Jill Stein,3
77389,ZAVALA3,Grn,Jill Stein,9
77390,ZAVALA4,Grn,Jill Stein,3
77391,ZAVALA4A,Grn,Jill Stein,0


In [495]:
# Viewing candidate data
general_data["candidate"].value_counts(dropna=False)

candidate
Gary Johnson                          6018
Jill Stein                            6007
Hillary Clinton                       6003
Donald Trump                          3707
Under Votes                           2744
                                      ... 
Michael a maturen/juan a munoz (w)       2
Mike Maturen                             1
Monica MooreHead/Lamont Lilly            1
Marco Rubio                              1
Jodey Arrington                          1
Name: count, Length: 210, dtype: int64

In [496]:
# Cleaning Candidates

# Turning all primary data to uppercase
general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*",
    r"\w*WRITE[\s-]\w*",
    r"NAN",
    r"UNCERTIFIED",
    r"UNVERIFIED",
    r"NONE OF THE ABOVE",
    r"LBT",
    r"DEM",
    r"REP",
    r"GRN",
    r"NAN"

]

pattern = "|".join(unwanted_keywords)

# FILTER OUT TRASH WORDS
general_data = general_data[~general_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

# Fixing Darrell Castle"
general_data.loc[
    general_data["candidate"].str.contains("\w*ASTLE\w*", case=False, na=False),
    "candidate"
] = "CASTLE"

general_data.loc[
    general_data["candidate"].str.contains("\w*ATUREN\w*", case=False, na=False),
    "candidate"
] = "MATUREN"

# Fixing mixed president + vice_president by /
general_data["candidate"] = (
    general_data["candidate"].str.split("/")
    .str[0]
    .str.strip()
    )

#Selecting only last name
general_data["candidate"] = general_data["candidate"].str.split().str[-1]

# Fixing the , candidate
# general_data["candidate"] = (
#     general_data["candidate"].str.split(",")
#     .str[0]
#     .str.strip()
#     )



# Fixing Cubbler
general_data.loc[
    ( 
        general_data["candidate"].str.contains("\w*UBBIER\w*", case=False, na=False)|
        general_data["candidate"].str.contains("\w*UBLER\w*", case=False, na=False)|
        general_data["candidate"].str.contains("\w*OBBLER\w*", case=False, na=False),
    "candidate")
] = "CUBBLER"


# Fixing Kotlikoff
general_data.loc[
    general_data["candidate"].str.contains("\w*TIKOFF\w*", case=False, na=False),
    "candidate"
] = "KOTLIKOFF"

# Fixing Valdivia
general_data.loc[
    general_data["candidate"].str.contains("\w*VALDIVA\W*", case=False, na=False),
    "candidate"
] = "VALDIVIA"

# Fixing HOEFLING
general_data.loc[
    general_data["candidate"].str.contains("\w*HOEFFLING", case=False, na=False),
    "candidate"
] = "HOEFLING"


# Fixing McMullin
general_data.loc[
    general_data["candidate"].str.contains("MCMULLEN", case=False, na=False) ,
    "candidate"
] = "MCMULLIN"

general_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\3597692450.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = general_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\3597692450.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = (
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\3597692450.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

candidate
TRUMP        8433
JOHNSON      8408
CLINTON      8390
STEIN        8366
MCMULLIN     4024
CASTLE       3376
MATUREN      2667
HOEFLING     2481
KOTLIKOFF    2433
VALDIVIA     2280
CUBBLER      2203
MOOREHEAD    2184
MORROW       2134
SOLTYSIK     1926
LEE          1867
STEFFES      1861
FOX          1675
HURD           18
GALLEGO        18
CORVALAN       18
BARAKA         14
SANDERS        11
CRUZ            8
SCHULIN         8
CHRISTIE        6
LEAMER          4
ERSKINE         4
LILLY           4
WALKER          4
CASE            4
BARRIERE        4
KUSHNER         4
RODRIGUEZ       4
RUBIO           1
ARRINGTON       1
Name: count, dtype: int64

In [497]:
# Viewing Party
general_data["party"].value_counts(dropna=False)

party
NaN             31654
REP              4260
LIB              4236
DEM              4218
GRN              4167
Republican       2351
Green            2349
Libertarian      2349
Democratic       2349
Dem              1627
Rep              1627
Grn              1627
Lbt              1574
WRITE-IN          104
D                  58
R                  58
L                  58
G                  58
Lib                52
ONA                30
wri                10
GEN                 6
Constitution        5
GREEN               4
LIBERTARIAN         4
DEMOCRATIC          4
REPUBLICAN          4
Name: count, dtype: int64

In [498]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown
    

#=====================================
# Function to fill remaining missing party from master lookup
#=====================================
def fill_party_from_master(df, master_df):
    party_map = master_df.set_index("candidate")["party"].to_dict()
    df["party"] = df.apply(
        lambda row: party_map.get(row["candidate"], row["party"])
        if pd.isna(row["party"]) else row["party"],
        axis=1
    )
    return df


#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)
    
    # Combine unique pairs
    new_data = df[["candidate", "party"]].dropna().drop_duplicates()
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
general_data["party"] = general_data.apply(
    lambda row: fill_party_from_data(row, general_data),
    axis=1
)

# Fill remaining party using general master CSV
# master_party_df = pd.read_csv(r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\master_candidate_party.csv") # USE YOUR OWN ADDRESS
#  general_data = fill_party_from_master(general_data, master_party_df)
general_data["party"].value_counts(dropna=False)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\1869924615.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["party"] = general_data.apply(


party
WRITE-IN        23727
REP              4260
LIB              4236
DEM              4218
GRN              4167
wri              4012
Constitution     3367
Republican       2351
Green            2349
Libertarian      2349
Democratic       2349
Dem              1779
Rep              1779
Grn              1771
Lbt              1574
Lib               205
None               66
D                  58
R                  58
L                  58
G                  58
ONA                30
GEN                 6
GREEN               4
LIBERTARIAN         4
DEMOCRATIC          4
REPUBLICAN          4
Name: count, dtype: int64

In [499]:
# Cleaning Party
# Turning all general data party to uppercase
general_data["party"] = general_data["party"].astype(str).str.upper()

general_data["party"] = (
    general_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GREEN",
        "LIBERTARIAN": "LIB",
        "LBT": "LIB",
        "L": "LIB",
        "CONSTITUTION": "CON",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK"
    })
)

general_data["party"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\477868490.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["party"] = general_data["party"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\477868490.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["party"] = (


party
IND      27775
REP       8452
LIB       8426
DEM       8408
GRN       8291
CON       3367
UNK         66
GREEN       58
Name: count, dtype: int64

In [500]:
# UPDATE MASTER FILE, CAREFUL
update_master_candidate_party(general_data, r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv")

In [501]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\1829951435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate_column"] = (
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3912\1829951435.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  general_result = general_data.pivot_table(


candidate_column,precinct,gen_con_CASTLE,gen_dem_CLINTON,gen_dem_GALLEGO,gen_green_STEIN,gen_grn_BARAKA,gen_grn_STEIN,gen_ind_CASTLE,gen_ind_CUBBLER,gen_ind_FOX,...,gen_unk_CRUZ,gen_unk_ERSKINE,gen_unk_KUSHNER,gen_unk_LEAMER,gen_unk_LILLY,gen_unk_RODRIGUEZ,gen_unk_RUBIO,gen_unk_SANDERS,gen_unk_SCHULIN,gen_unk_WALKER
0,ANDERSON1,1,262,0,0,0,5,0,0.0,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0
1,ANDERSON10,0,37,0,0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0
2,ANDERSON11,0,130,0,0,0,2,0,0.0,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0
3,ANDERSON13-14,0,268,0,0,0,7,0,0.0,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0
4,ANDERSON15,0,84,0,0,0,1,0,0.0,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8299,ZAVALA2,0,764,0,0,0,3,0,0.0,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0
8300,ZAVALA3,0,737,0,0,0,9,0,0.0,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0
8301,ZAVALA4,0,353,0,0,0,3,0,0.0,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0
8302,ZAVALA4A,0,237,0,0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0.0,0.0,0,0


In [504]:
# Merge
combined = pd.merge(primary_result, general_result, on="precinct", how="inner")

# Convert DEM primary columns to numeric and calculate total
dem_cols = combined.filter(like="pri_dem_").columns
combined[dem_cols] = combined[dem_cols].apply(pd.to_numeric, errors="coerce")
combined["dem_primary_total"] = combined[dem_cols].sum(axis=1)

# Convert general election columns to numeric and calculate total
gen_cols = combined.filter(like="gen_").columns
combined[gen_cols] = combined[gen_cols].apply(pd.to_numeric, errors="coerce")
combined["general_total"] = combined[gen_cols].sum(axis=1)

# Convert all numeric columns to int
for col in combined.columns[1:]:
    combined[col] = pd.to_numeric(combined[col], errors='coerce').fillna(0).astype(int)

# Save combined
combined.to_csv("TX.csv", index=False)

# Identify filtered-out precincts
primary_precincts = set(primary_result["precinct"])
general_precincts = set(general_result["precinct"])
combined_precincts = set(combined["precinct"])

primary_filtered_out = primary_precincts - combined_precincts
general_filtered_out = general_precincts - combined_precincts




In [505]:
# TX lo, AZ lo

combined.to_csv("TX.csv", index=False)
# Step 3: Identify unmatched precincts and retrieve full rows
combined_precincts = set(combined["precinct"])

# Rows in primary_result but not in combined
primary_filtered_out = primary_result[~primary_result["precinct"].isin(combined_precincts)]
primary_filtered_out.to_csv("TX_primary_filtered.csv", index=False)

# Rows in general_result but not in combined
general_filtered_out = general_result[~general_result["precinct"].isin(combined_precincts)]
general_filtered_out.to_csv("TX_general_filtered.csv", index=False)